In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
train = pd.read_csv('../input/tmdb-box-office-prediction/train.csv', index_col=0)
train.info()

In [ ]:
test = pd.read_csv('../input/tmdb-box-office-prediction/test.csv', index_col=0)
test.info()

In [ ]:
df = pd.concat([train, test])

In [ ]:
df["log_revenue"] = np.log10(df["revenue"])

In [ ]:
train.head().loc[:, :"poster_path"]

In [ ]:
train.head().loc[:, "production_companies":]

In [ ]:
# 使わない列を消す
df = df.drop(["imdb_id", "poster_path", "status", "overview", "original_title"], axis=1)

In [ ]:
# homepage は、有るか無いかにする
df["homepage"] = ~df["homepage"].isnull()

In [ ]:
import re
import ast

## 各列の処理

In [ ]:
dfdic_feature = {}

### genre

In [ ]:
# 文字列から、"name" 情報を抽出しリストに
def to_name_list(text):
    txt_list = re.sub("[\[\]]", "", text).replace("}, {", "}|{").split("|")
    return [ ast.literal_eval(txt)["name"] for txt in txt_list ]

def to_id_list(text):
    txt_list = re.sub("[\[\]]", "", text).replace("}, {", "}|{").split("|")
    return [ ast.literal_eval(txt)["id"] for txt in txt_list ]

In [ ]:
df["genre_names"] = df["genres"].fillna("[{'name': 'nashi'}]").map(to_name_list)

In [ ]:
# 各ワードの有無を表す 01 のデータフレームを作成
def count_word_list(series):
    len_max = series.apply(len).max() # ジャンル数の最大値
    tmp = series.map(lambda x: x+["nashi"]*(len_max-len(x))) # listの長さをそろえる
    
    word_set = set(sum(list(series.values), [])) # 全ジャンル名のset
    for n in range(len_max):
        word_dfn = pd.get_dummies(tmp.apply(lambda x: x[n]))
        word_dfn = word_dfn.reindex(word_set, axis=1).fillna(0).astype(int)
        if n==0:
            word_df = word_dfn
        else:
            word_df = word_df + word_dfn
    
    return word_df.drop("nashi", axis=1)

In [ ]:
dfdic_feature["genre"] = count_word_list(df["genre_names"])

In [ ]:
# 各ジャンルの作品数
dfdic_feature["genre"].sum().sort_values(ascending=False)

### original_language

In [ ]:
# 各言語の作品数
df["original_language"].value_counts()

In [ ]:
# train内の作品数が10件未満の言語は "small" に集約
n_language = df.loc[:train.index[-1], "original_language"].value_counts()
small_language = n_language[n_language<10].index
df.loc[df["original_language"].isin(small_language), "original_language"] = "small"

In [ ]:
# one_hot_encoding
dfdic_feature["original_language"] = pd.get_dummies(df["original_language"])
dfdic_feature["original_language"].head()

### production_companies

In [ ]:
df["production_companies"]

In [ ]:
# 複数社で製作のケースもある
df["production_companies"][1]

In [ ]:
df["production_names"] = df["production_companies"].fillna("[{'name': 'nashi'}]").map(to_name_list)

In [ ]:
tmp = count_word_list(df["production_names"])

In [ ]:
# 順位:作品数
tmp.sum().sort_values(ascending=False).reset_index(drop=True).plot(loglog=True)
plt.xlabel("rank")
plt.ylabel("number of movies")

In [ ]:
tmp.loc[:3000].sum().sort_values(ascending=False).head(20)

In [ ]:
# train内の件数が多い物のみ選ぶ
def select_top_n(df, topn=9999, nmin=2):  # topn:上位topn件, nmin:作品数nmin以上
    if "small" in df.columns:
        df = df.drop("small", axis=1)
    n_word = (df.loc[train.index]>0).sum().sort_values(ascending=False)
    # 作品数がnmin件未満
    smallmin = n_word[n_word<nmin].index
    # 上位topn件に入っていない
    smalln = n_word.iloc[topn+1:].index
    small = set(smallmin) | set(smalln)
    # 件数の少ないタグのみの作品
    df["small"] = (df[small].sum(axis=1)>0)*1
    
    return df.drop(small, axis=1)

In [ ]:
# trainに50本以上作品のある会社
dfdic_feature["production_companies"] = select_top_n(tmp, nmin=50)
dfdic_feature["production_companies"].head()

In [ ]:
# revenue との相関
dfdic_feature["production_companies"].corrwith(df["log_revenue"]).sort_values()
# 全ての会社が正の相関。 ->　欠測だと低い。多数の会社が参加すると高い?

### production_countries

In [ ]:
df["production_countries"]

In [ ]:
# 複数国のケース
df["production_countries"][7394]

In [ ]:
# 欠損
df["production_countries"].isnull().sum()

In [ ]:
# 国名のリストに
df["country_names"] = df["production_countries"].str.replace("United States of America", "USA").fillna("[{'name': 'nashi'}]").map(to_name_list)

In [ ]:
df_country = count_word_list(df["country_names"])

In [ ]:
# 国別製作本数ランキング
df_country.sum().sort_values(ascending=False).head(10)

In [ ]:
df_country.sum().sort_values(ascending=False).reset_index(drop=True).plot(loglog=True)
plt.xlabel("rank")
plt.ylabel("number of movies")

In [ ]:
# 2か国だったら、0.5ずつに
df_country = (df_country.T/df_country.sum(axis=1)).T.fillna(0)

In [ ]:
df_country = select_top_n(df_country, nmin=30)

In [ ]:
dfdic_feature["production_countries"] = df_country

In [ ]:
dfdic_feature["production_countries"].head()

In [ ]:
# revenue との相関
dfdic_feature["production_countries"].corrwith(df["log_revenue"]).sort_values()


### Keywords

In [ ]:
df["Keywords"]

In [ ]:
df["keyword_list"] = df["Keywords"].fillna("[{'name': 'nashi'}]").map(to_name_list)

In [ ]:
# キーワードが149個ある作品も
df["keyword_list"].apply(len).max()

In [ ]:
# 全キーワードの種類
keyword_set = set(sum(list(df["keyword_list"].values), []))
len(keyword_set)

In [ ]:
import collections


In [ ]:
# 多いキーワードtop20
keyword_count = pd.Series(collections.Counter(sum(list(df["keyword_list"].values), [])))
keyword_count = keyword_count.sort_values(ascending=False)
keyword_count.head(20)

In [ ]:
keyword_count.sort_values(ascending=False).reset_index(drop=True).plot(loglog=True)
plt.xlabel("rank")
plt.ylabel("number of movies")

In [ ]:
# keyword上位100
keyword_count.iloc[:101]
# nashi は欠損を置き換えたもの

In [ ]:
df_keyword = df[[]].copy()
# 上位１００件のキーワードのみ
for word in keyword_count.drop("nashi").iloc[:100].index:
    df_keyword[word] = df["keyword_list"].apply(lambda x: word in x)*1


In [ ]:
dfdic_feature["Keywords"] = df_keyword

In [ ]:
df_keyword.head()

In [ ]:
# revenue との相関
dfdic_feature["Keywords"].corrwith(df["log_revenue"]).sort_values()

### spoken_languages

多くの場合、original_langage と被る。  
使われている原語の数のみ入れてみる。

In [ ]:
df[["original_language","spoken_languages"]].head(30)

In [ ]:
def to_tag_list(text, tag):
    txt_list = re.sub("[\[\]]", "", text).replace("}, {", "}|{").split("|")
    return [ ast.literal_eval(txt)[tag] for txt in txt_list ]

# df["language_names"] = df["spoken_languages"].fillna("[{'iso_639_1': 'nashi'}]").apply(to_tag_list, tag = 'iso_639_1')
# df_spklanguage = count_word_list(df["language_names"])

In [ ]:
df["language_names"] = df["spoken_languages"].fillna("[{'iso_639_1': 'nashi'}]").apply(to_tag_list, tag = 'iso_639_1')
# 欠損値は１になる
df["n_language"] = df["language_names"].apply(len)

In [ ]:
# revenue との相関
df["n_language"].corr(df["log_revenue"])

### belongs_to_collection

In [ ]:
# 複数のシリーズに属していることはない
assert df["belongs_to_collection"].fillna("[{'name': ''}]").map(to_name_list).map(len).max()==1

In [ ]:
df["collection_name"] = df["belongs_to_collection"].fillna("[{'name': 'nashi'}]").map(to_name_list).map(lambda x: x[0])

In [ ]:
df["collection_name"].value_counts().head(10)

In [ ]:
dfdic_feature["belongs_to_collection"] = pd.get_dummies(df["collection_name"])

In [ ]:
# revenue との相関
dfdic_feature["belongs_to_collection"].corrwith(df["log_revenue"]).sort_values().dropna()

#### シリーズ毎の平均log(revenue)

In [ ]:
collection_av = df.groupby("collection_name").mean()[["log_revenue"]].dropna()
collection_av.columns = ["collection_av_logrevenue"]

In [ ]:
collection_av.sort_values("collection_av_logrevenue", ascending=False)

In [ ]:
df = df.merge(collection_av, on="collection_name", how="left")
nashi_mean = df.loc[df["collection_name"]=="nashi","collection_av_logrevenue"].mean()
df["collection_av_logrevenue"] = df["collection_av_logrevenue"].fillna(nashi_mean)  # train に無いシリーズの場合、シリーズ無しと同じにする
#df = df.rename(columns={"log_revenue":"collection_mean_log_revenue"})

## 結合

In [ ]:
df_features = pd.concat(dfdic_feature, axis=1)

In [ ]:
df_features.shape

In [ ]:
df_features.head()

In [ ]:
df_features.isnull().sum().sum()

In [ ]:
df.isnull().sum()

## 欠測処理

In [ ]:
# 平均で埋める
df["runtime"] = df["runtime"].fillna(df["runtime"].mean())

In [ ]:
df[df["release_date"].isnull()]["title"]

In [ ]:
# May,2000 (https://www.imdb.com/title/tt0210130/) 
# 日は不明。1日を入れておく
df.loc[3828, "release_date"] = "5/1/00"

## 日付

In [ ]:
df["release_date"]

In [ ]:
df["release_year"] = pd.to_datetime(df["release_date"]).dt.year
df.loc[df["release_year"]>2020, "release_year"] = df.loc[df["release_year"]>2020, "release_year"]-100

df["release_month"] = pd.to_datetime(df["release_date"]).dt.month
df["release_day"] = pd.to_datetime(df["release_date"]).dt.day

df["release_dayofyear"] = pd.to_datetime(df["release_date"]).dt.dayofyear
df["release_dayofweek"] = pd.to_datetime(df["release_date"]).dt.dayofweek

In [ ]:
df.groupby("release_year").mean()["revenue"].plot()
plt.ylabel("mean revenie")

In [ ]:
df.groupby("release_year").mean()["log_revenue"].plot()
plt.ylabel("mean log(revenie)")

In [ ]:
df.groupby("release_month").mean()["log_revenue"].plot()
plt.ylabel("mean log(revenie)")

In [ ]:
df.groupby("release_dayofyear").mean()["log_revenue"].plot()
plt.ylabel("mean log(revenie)")

In [ ]:
df.groupby("release_dayofweek").mean()["log_revenue"].plot()
plt.ylabel("mean log(revenie)")

In [ ]:
df.groupby("release_day").mean()["log_revenue"].plot()
plt.ylabel("mean log(revenie)")

In [ ]:
df.isnull().sum()

In [ ]:
df["budget"]

In [ ]:
df["release_month"] = df["release_month"].astype('category')
df["release_dayofweek"] = df["release_dayofweek"].astype('category')

### budget=0 の扱い

In [ ]:
plt.scatter(df["budget"]+1, df["log_revenue"], s=1)
plt.xscale("log")
#plt.xrange([])

In [ ]:
df.loc[df["budget"]==0, "log_revenue"].hist()
plt.xlabel("log(revenue)")
plt.ylabel("number")

In [ ]:
df.isnull().sum()

In [ ]:
df[["original_language", "collection_name"]] = df[["original_language", "collection_name"]].astype("category")

In [ ]:
df.columns

In [ ]:
df_use = df[['budget', 'homepage', 'popularity','runtime','n_language', 'collection_av_logrevenue',
             'release_year', 'release_month','release_dayofweek']]
df_use

In [ ]:
df_use = pd.get_dummies(df_use)

## 学習用データ作成

In [ ]:
df_use.index = df_features.index

In [ ]:
df_input = pd.concat([df_use, df_features.drop("belongs_to_collection", axis=1)], axis=1)

In [ ]:
df_input.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 


In [ ]:
df["ln_revenue"] = np.log(df["revenue"]+1)

In [ ]:
train_X, val_X, train_y, val_y, train_rev, val_rev = train_test_split(df_input[:3000], 
                                                  df.loc[:3000, "ln_revenue"], 
                                                  df.loc[:3000, "revenue"], 
                                                  test_size=0.25)

## Lasso

In [ ]:
from sklearn.linear_model import Lasso, Ridge

In [ ]:
clf = Lasso(alpha=0.1)  # default alpha=1
clf.fit(train_X, train_y)

In [ ]:
coef = pd.Series(clf.coef_, index=train_X.columns)
coef[coef!=0]

In [ ]:
val_pred = clf.predict(val_X)
np.sqrt(mean_squared_error(val_pred, val_y))